In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

#DATA

In [ ]:
DATADIR = '/content/gdrive/MyDrive/Case_1/UCI HAR Dataset'


In [ ]:
#Own
#print(DATADIR)

/content/gdrive/MyDrive/Case_1/UCI HAR Dataset


In [ ]:
SIGNALS = [
           "body_acc_x",
           "body_acc_y",
           "body_acc_z",
           "body_gyro_x",
           "body_gyro_y",
           "body_gyro_z",
           "total_acc_x",
           "total_acc_y",
           "total_acc_z"
]

In [ ]:
for signal in SIGNALS:
    print(signal)

body_acc_x
body_acc_y
body_acc_z
body_gyro_x
body_gyro_y
body_gyro_z
total_acc_x
total_acc_y
total_acc_z


In [ ]:
def _read_csv(filename):
    a = pd.read_csv(filename, delim_whitespace=True, header = None)
    return a

def load_signals(subset):
    signals_data = []
    for signal in SIGNALS:
        filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).to_numpy()
        )

    return np.transpose(signals_data, (1,2,0))


In [ ]:
#Own

#for signal in SIGNALS:
#    filename = f'{DATADIR}/{"train"}/Inertial Signals/{signal}_{"train"}.txt'
#    print(filename)

/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_acc_x_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_acc_y_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_acc_z_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_gyro_x_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_gyro_y_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/body_gyro_z_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/total_acc_x_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/total_acc_y_train.txt
/content/gdrive/MyDrive/Case_1/UCI HAR Dataset/train/Inertial Signals/total_acc_z_train.txt


In [ ]:
#Own
#alamin = _read_csv(filename).to_numpy()

(7352, 128)


In [ ]:
print(alamin)

[[ 0.1029341   0.1056872   0.1021025  ...  0.1003852   0.09987355
   0.09498741]
 [ 0.09792958  0.09935086  0.09811381 ...  0.0935352   0.08903516
   0.09061235]
 [ 0.09111667  0.0926756   0.09606378 ...  0.08301135  0.08233391
   0.08148748]
 ...
 [ 0.1745865   0.2324243   0.2576066  ...  0.1112952   0.1263023
   0.1886212 ]
 [ 0.262814    0.2954154   0.3376098  ...  0.1289247   0.1722512
   0.2100712 ]
 [ 0.2093865   0.2310711   0.3341906  ... -0.156105   -0.1227979
  -0.08357159]]


In [ ]:
def load_y(subset):
    filename = f'{DATADIR}/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]
    print(y)
    print(type(y))
    return pd.get_dummies(y).to_numpy()

In [ ]:
#Own
#filename = f'{DATADIR}/{"train"}/y_{"train"}.txt'
#print(filename)

In [ ]:
def load_data():
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')
    return X_train, X_test, y_train, y_test

In [ ]:
np.random.seed(42) # Random_State
import tensorflow.compat.v1 as tf
tf.set_random_seed(4)

In [ ]:
#Own
#print(tf.set_random_seed(4))

None


In [ ]:
#Use bese core
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads = 1
)

In [ ]:
from tensorflow.compat.v1.keras.backend import set_session
sess = tf.Session(graph= tf.get_default_graph(), config = session_conf)
set_session(sess)

#Model

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

epochs = 30
batch_size = 16
n_hidden = 32

In [ ]:
# Count the number of classes

def _count_classes(y):
    return len(set(tuple(category) for category in y))

In [ ]:
#print(len(set(tuple(category) for category in y_train)))

In [ ]:
X_train, X_test, y_train, y_test = load_data()

0       5
1       5
2       5
3       5
4       5
       ..
7347    2
7348    2
7349    2
7350    2
7351    2
Name: 0, Length: 7352, dtype: int64
<class 'pandas.core.series.Series'>
0       5
1       5
2       5
3       5
4       5
       ..
2942    2
2943    2
2944    2
2945    2
2946    2
Name: 0, Length: 2947, dtype: int64
<class 'pandas.core.series.Series'>


In [ ]:
#print(y_train)

[[0 0 0 0 1 0]
 [0 0 0 0 1 0]
 [0 0 0 0 1 0]
 ...
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]
 [0 1 0 0 0 0]]


In [ ]:
X_train.shape

(7352, 128, 9)

In [ ]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

In [ ]:
print(timesteps)
print(input_dim)
print(n_classes)

128
9
6


In [ ]:
y_test

array([[0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 1, 0],
       ...,
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0]], dtype=uint8)

#LSTM Model

In [ ]:
model = Sequential()
model.add(LSTM(64,input_shape=(timesteps,input_dim),return_sequences=True))
model.add(LSTM(64,return_sequences=True))
model.add(LSTM(64))
model.add(Dense(n_classes,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128, 64)           18944     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128, 64)           33024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 6)                 390       
Total params: 85,382
Trainable params: 85,382
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = batch_size, epochs = epochs, validation_data = (X_test, y_test))

Epoch 1/30
 21/460 [>.............................] - ETA: 1:13 - loss: 1.5286 - accuracy: 0.3232

KeyboardInterrupt: ignored